In [ ]:
from typing import Dict, Union, Literal, Tuple, Generator, List
from pydantic import BaseModel
import tokenize
from io import StringIO
import inspect


class _Tokenize:
    @classmethod
    def _get_tokens(cls, function_block: str):
        tokens = tokenize.generate_tokens(StringIO(function_block).readline)
        return tokens

    @classmethod
    def get_token_info(cls, function_block):
        tokens = cls._get_tokens(function_block)
        items = []
        for toknum, tokval, start, end, _ in tokens:
            items.append({
                'Type': tokenize.tok_name[toknum],
                'Value': tokval,
                'Start Line': start[0],
                'Start Column': start[1],
                'End Line': end[0],
                'End Column': end[1]
            })
        return items